In [212]:
from google.colab import auth
import gspread
from google.auth import default
import pandas as pd
import numpy as np
from google.cloud import bigquery
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder

**Finding the Z-Score**

In [213]:
client = bigquery.Client.from_service_account_json('/content/drive/MyDrive/tech-cali-b2c-72b3e690e309-Compute-Engine.json')

project_id = 'tech-cali-b2c'
dataset_id = 'CE_External_Source'
table_id = 'Chartmetric_Raw'

# Create a reference to the BigQuery table
table_ref = client.dataset(dataset_id).table(table_id)

df_mod = client.query(f"SELECT * FROM `{project_id}.{dataset_id}.{table_id}`").to_dataframe()

In [214]:
# df = pd.DataFrame.from_records(rows[1:],columns=rows[0])

df_mod = df_mod.drop_duplicates(subset=['artist_id'], keep='first')
df_mod = df_mod[df_mod['artist_id'] != ""]

In [ ]:
df = df_mod.drop(['Snapshot_Date'], axis=1)

In [215]:
# df.iloc[:,3:] = df.iloc[:,3:].replace("",0)
df.iloc[:,3:] = df.iloc[:,3:].fillna(0)

In [217]:
cols = ['name', 'country_code', 'genres', 'artist_id']
columns_select = list(df.drop(cols, axis=1).columns)
for col in columns_select:
    df[col] = df[col].astype('float')

In [219]:
exclude = ['name', 'country_code', 'genres', 'artist_id', 'youtube_daily_video_views','youtube_channel_views',
           'TikTok_Genz_followers_percentage','Instagram_Genz_followers_percentage','TikTok_number_of_top_tracks',
           'TikTok_avg_creations_for_top_n_tracks','TikTok_avg_views_for_top_n_tracks', 'Youtube_Genz_subscribers_percentage']
cols = [x for x in list(df.columns) if x not in exclude]

for col in cols:
    col_zscore = col + '_zscore'
    df[col_zscore] = (df[col] - df[col].median())/df[col].std(ddof=0)

In [221]:
def map_score_bucket(zscore):
  if zscore < -2:
    return 1
  elif zscore >= -2 and zscore < -1:
    return 2
  elif zscore >= -1 and zscore < 0:
    return 3
  elif zscore >= 0 and zscore < 0.5:
    return 4
  elif zscore >= 0.5 and zscore < 1:
    return 5
  elif zscore >= 1 and zscore < 2:
    return 6
  elif zscore >= 2 and zscore < 3:
    return 7
  elif zscore >= 3:
    return 8
  else:
    return 0

In [222]:
import math

def map_score_bucket_label(score_bucket):
  if math.ceil(score_bucket) >= 8:
    return "8.Extremely High"
  elif math.ceil(score_bucket) >= 7:
    return "7.Very High"
  elif math.ceil(score_bucket) >= 6:
    return "6.High"
  elif math.ceil(score_bucket) >= 5:
    return "5.Above Medium"
  elif math.ceil(score_bucket) >= 4:
    return "4.Medium"
  elif math.ceil(score_bucket) >= 3:
    return "3.Below Medium"
  elif math.ceil(score_bucket) >= 2:
    return "2.Low"
  elif math.ceil(score_bucket) >= 1:
    return "1.Very Low"
  else:
    return "0.Not Applicable"

In [223]:
for col in list(df.columns):
  if col.endswith('_zscore'):
    col_score_bucket = "_".join(col.split('_')[:-1]) + '_z_score'
    col_score_bucket_label = col_score_bucket + '_label'
    df[col_score_bucket] = df[col].apply(lambda x: map_score_bucket(x))
    df[col_score_bucket_label] = df[col_score_bucket].apply(lambda x: map_score_bucket_label(x))

In [ ]:
from google.oauth2 import service_account
import pandas_gbq

# Define your BigQuery project ID and credentials
project_id = 'tech-cali-b2c'
credentials = service_account.Credentials.from_service_account_file('/content/drive/MyDrive/tech-cali-b2c-72b3e690e309-Compute-Engine.json')

# Define the BigQuery table name and dataset ID
dataset_id = 'CE_ML_Layer'
table_name = 'Z_Scoring'

# Write the dataframe to BigQuery
pandas_gbq.to_gbq(df, f'{dataset_id}.{table_name}', project_id=project_id, if_exists='replace', credentials=credentials)

print('Data moved to BigQuery successfully!')